In [1]:
# selenium 4
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import numpy as np
from selenium.webdriver.common.action_chains import ActionChains

strcutrue of our table

In [331]:
df = pd.DataFrame(columns=[
    'City', 'Interval', 'LCR', 'PGR', 'LCRPGR',
    'City Definition data', 'Built-up data', 'Population data',
    'Within city definition T1 Built-up (km²)', 'Within city definition T1 Population (*10,000)','Within city definition T1 LCPC (m²/person)',
    'Within city definition T2 Built-up (km²)', 'Within city definition T2 Population (*10,000)','Within city definition T2 LCPC (m²/person)',
    '% Change in LCPC LCPC (m²/person)', 'Built-up T1', 'Built-up T2'
])

定义excel文件路径

In [335]:
save_file_path='citylevel_GHSPop1km_2020_2015_GHSPop100m.xlsx'

打开 sdg indicator 页面

In [183]:
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))
# Now you can use the driver to interact with the web page
driver.get("https://gisproject-1.projects.earthengine.app/view/sdg-1131-indicator-v7")

original_window_handle记录的是sdg indicator主页面的参数，后面通过页面跳转爬取数据后可以再跳回主页面

In [184]:
# Get the handle of the original window
original_window_handle = driver.current_window_handle
print(original_window_handle)

D642905659FB2383E8B8C5E5096F4422


这一步是让页面点击city下拉框，然后程序可以获取所有城市选项；  
输出城市列表的index和城市名的目的：后面程序中途崩了可以需要重新开始跑的话，可以跳过已经爬好的城市。  
  
如何跳过爬好的城市：  
1. 修改enumerate(city_options[0:],start=index_in_df)中city_options[:]和index_in_df的值
2. city_options[0:]中的0修改成要下一个要跑的城市的index，城市index是指从页面下拉框中获取的城市列表中第几个，从0开始算
3. index_in_df表示从表格哪一行开始存数据，所以要确认它的值更新为表格中下一行开始存数据的行数，从0开始算。

In [336]:
driver.switch_to.window(original_window_handle)
##############################################################
# Select City
##############################################################
# Find the element using its CSS selector
city_element = driver.find_elements(By.CSS_SELECTOR, '#content > div > div.ui-widget.ui-panel.stretch-horizontal.stretch-vertical > div > div.ui-widget.custom-width.ui-panel > div > div:nth-child(3) > div > div:nth-child(2) > div > div.ui-widget.ui-select > div')[0]
# print(city_element.get_attribute("aria-posinset"))
# # Use JavaScript to modify the aria-posinset attribute value of the element
# new_posinset_value = "Beijing"  # Replace with the desired new value
# driver.execute_script("arguments[0].innerText = arguments[1];", city_element, new_posinset_value)
city_element.click()
city_options = WebDriverWait(driver, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, 'goog-menuitem-content'))
)

In [337]:
for index,city in enumerate(city_options):
  print(index,city.text)

0 Stockholm
1 Gothenburg
2 Abu Dhabi
3 Accra
4 Batam
5 Beijing
6 Buenos Aires
7 Cairo
8 Charleston
9 Dar Es Salaam
10 Detroit
11 Dodoma
12 Dubai
13 Gombe
14 Guangzhou
15 Halle
16 Heidelberg
17 Islamabad
18 Johannesburg
19 Kampala
20 Karachi
21 Katowice
22 Kigali
23 Kitchener
24 Kolkata
25 Lagos
26 La Paz
27 Lilongwe
28 Luanda
29 Manama
30 Mexico City
31 Mogadishu
32 Mumbai
33 Nairobi
34 New York
35 Niamey
36 Nouakchott
37 Ouagadougou
38 Quito
39 Rio de Janeiro
40 Shanghai
41 Sydney
42 Tehran
43 Tianjin
44 Xiamen
45 Yinchuan
46 
47 
48 
49 
50 
51 
52 
53 
54 
55 
56 
57 
58 
59 
60 
61 
62 
63 
64 
65 
66 
67 


In [ ]:
# Find the element using its CSS selector

city_element = driver.find_elements(By.CSS_SELECTOR, '#content > div > div.ui-widget.ui-panel.stretch-horizontal.stretch-vertical > div > div.ui-widget.custom-width.ui-panel > div > div:nth-child(3) > div > div:nth-child(2) > div > div.ui-widget.ui-select > div')[0]

builtup_element = driver.find_elements(By.CSS_SELECTOR, '#content > div > div.ui-widget.ui-panel.stretch-horizontal.stretch-vertical > div > div.ui-widget.custom-width.ui-panel > div > div:nth-child(4) > div > div:nth-child(3) > div > div > div > div:nth-child(1) > div > div.ui-widget.ui-select > div')[0]

population_element = driver.find_elements(By.CSS_SELECTOR, '#content > div > div.ui-widget.ui-panel.stretch-horizontal.stretch-vertical > div > div.ui-widget.custom-width.ui-panel > div > div:nth-child(4) > div > div:nth-child(3) > div > div > div > div:nth-child(3) > div > div.ui-widget.ui-select > div')[0]

定义从表格哪一行开始存数据

In [338]:
index_in_df=0

下面的循环就开始爬数据了，现在是被修改成能循环城市和builtup数据集，如果还需要循环population数据集的话，可以到参考最下面的代码在循环builtup数据集后加一层population数据集的循环

In [340]:
##############################################################
# interate City
##############################################################
for index, city in enumerate(city_options[0:],start=index_in_df):
  if city.text=='':
        continue
  city_name = city.text
  print(f"Clicking on {city_name} - index:{index}")

  # 点击城市
  city.click()

  ##############################################################
  # generate city definition
  ##############################################################
  # press the generate city definition buttons
  generate_city_definition_T2_element=driver.find_elements(By.CSS_SELECTOR, '#content > div > div.ui-widget.ui-panel.stretch-horizontal.stretch-vertical > div > div.ui-widget.custom-width.ui-panel > div > div:nth-child(3) > div > div:nth-child(12) > div > div:nth-child(1) > div')[0]
  generate_city_definition_T1_element=driver.find_elements(By.CSS_SELECTOR, '#content > div > div.ui-widget.ui-panel.stretch-horizontal.stretch-vertical > div > div.ui-widget.custom-width.ui-panel > div > div:nth-child(3) > div > div:nth-child(12) > div > div:nth-child(2) > div')[0]
  generate_city_definition_T2_element.click()
  generate_city_definition_T1_element.click()

  # give the appliciton some time to generate the definitions
  time.sleep(20)

  ##############################################################
  # iterate builtup dataset
  ##############################################################
  builtup_element.click()
  builtup_options = WebDriverWait(driver, 10).until(
      EC.presence_of_all_elements_located((By.CLASS_NAME, 'goog-menuitem-content')))  

  for index, builtup in enumerate(builtup_options[0:],start=0):
    if builtup.text=='':
        continue
    if builtup.text=='Own data':
        builtup.click()
        break
    print(f'Clicking on builtup {builtup.text} - index: {index}')
    builtup.click()

    ##############################################################
    # compute indicators
    ##############################################################
    #click the compute button
    compute_element=driver.find_elements(By.CSS_SELECTOR, '#content > div > div.ui-widget.ui-panel.stretch-horizontal.stretch-vertical > div > div.ui-widget.custom-width.ui-panel > div > div:nth-child(4) > div > div:nth-child(4) > div > div > div > div.ui-widget.ui-button > div')[0]
    compute_element.click()
    time.sleep(8)

    ##############################################################
    # collect the indicators
    ##############################################################
    # go through each chart in a new window
    pop_out_icons = driver.find_elements(By.CLASS_NAME,"pop-out-icon")  
    # print(len(pop_out_icons))
    #click the pop put icon
    for icon in pop_out_icons:
      icon.click()

    # Assuming now have four windows (the original one and the new three)
    window_handles = driver.window_handles
    # print(window_handles)

    # Switch to the new window or tab
    new_window_handle = [handle for handle in window_handles if handle != original_window_handle]

    for i,w in enumerate(new_window_handle):
      driver.switch_to.window(w)   
      cityname=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div > table > tbody > tr > td:nth-child(1)')
      LCPC=driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(3) > text:nth-child(1)')
      CIUI=driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(3) > text:nth-child(1)')
      
      # different charts
      #primary
      if cityname:
        # print('Print values with sleep')
        values=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div > table > tbody > tr>td')
        keys=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div > table > thead > tr>th')      
        for k,v in zip(keys,values):
          df.at[index_in_df,k.text]=v.text

      else:
        #LCPC
        if 'Land Consumption per Capita' in LCPC[0].text:  
          rectangles = driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(5) > g:nth-child(2) > g:nth-child(1)>rect')
          rectangles = driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(5) > g:nth-child(2) > g:nth-child(1)>rect')
          # print(len(rectangles))
          for r in rectangles:
          # 使用ActionChains进行悬停操作
            actions = ActionChains(driver)  
            actions.move_to_element(r).perform()
            # 等待一段时间
            time.sleep(2)  # 这里的1表示等待1秒，你可以根据需要调整时间 
            texts=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(7) > g > g > text')
            # print(len(texts))
            # for t in texts:
            #   print(t.text)
            if len(texts)>0:
              df.at[index_in_df,texts[1].text[:-1]+' '+texts[0].text]=texts[2].text
            
        #CIUI
        # if "Change in Urban Infill" in CIUI[0].text:
        else:
          rectangles = driver.find_elements(By.CSS_SELECTOR, '#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(5) > g:nth-child(2) > g:nth-child(1)>rect')
          # print(len(rectangles))
          for r in rectangles:
          # 使用ActionChains进行悬停操作
            actions = ActionChains(driver)  
            actions.move_to_element(r).perform()
            # 等待一段时间
            time.sleep(2)  # 这里的1表示等待1秒，你可以根据需要调整时间 
            texts=driver.find_elements(By.CSS_SELECTOR,'#chart-container > div > div:nth-child(1) > div > svg > g:nth-child(7)> g > g > text')
            # print(len(texts))
            # for t in texts:
            #   print(t.text)
            if len(texts)>0:
              df.at[index_in_df,texts[1].text[:-1]]=texts[2].text
      time.sleep(3)

    
    # 关闭所有新窗口
    for w in new_window_handle:      
        driver.switch_to.window(w)
        driver.close()
    #swith to the origin main window
    driver.switch_to.window(original_window_handle)

    index_in_df+=1
    # 保存DataFrame到Excel文件
    df.to_excel(save_file_path, index=False)
      
    ##############################################################
    # Click builtup
    ##############################################################
    builtup_element.click()

  ##############################################################
  # Select City
  ##############################################################
  city_element.click()
  city_options = WebDriverWait(driver, 10).until(
      EC.presence_of_all_elements_located((By.CLASS_NAME, 'goog-menuitem-content'))
  )

Clicking on Stockholm - index:0
Clicking on builtup GHS-Built 30m - index: 68
Clicking on builtup WSF evolution - index: 69
Clicking on builtup GHS_Built_S 100m - index: 70
Clicking on Gothenburg - index:1
Clicking on builtup GHS-Built 30m - index: 64
Clicking on builtup WSF evolution - index: 65
Clicking on builtup GHS_Built_S 100m - index: 66
Clicking on Abu Dhabi - index:2
Clicking on builtup GHS-Built 30m - index: 64
Clicking on builtup WSF evolution - index: 65
Clicking on builtup GHS_Built_S 100m - index: 66
Clicking on Accra - index:3
Clicking on builtup GHS-Built 30m - index: 64
Clicking on builtup WSF evolution - index: 65
Clicking on builtup GHS_Built_S 100m - index: 66
Clicking on Batam - index:4
Clicking on builtup GHS-Built 30m - index: 64
Clicking on builtup WSF evolution - index: 65
Clicking on builtup GHS_Built_S 100m - index: 66
Clicking on Beijing - index:5
Clicking on builtup GHS-Built 30m - index: 64
Clicking on builtup WSF evolution - index: 65


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=120.0.6099.71); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00676E73+174291]
	(No symbol) [0x005A0AC1]
	(No symbol) [0x002B6FF6]
	(No symbol) [0x002BAEAC]
	(No symbol) [0x002BC585]
	(No symbol) [0x002BC622]
	(No symbol) [0x002EDDAA]
	(No symbol) [0x002ED674]
	(No symbol) [0x00320C5C]
	(No symbol) [0x0030700C]
	(No symbol) [0x0031A104]
	(No symbol) [0x00306DA6]
	(No symbol) [0x002E1034]
	(No symbol) [0x002E1F8D]
	GetHandleVerifier [0x007149CC+820268]
	sqlite3_dbdata_init [0x007D4EBE+652494]
	sqlite3_dbdata_init [0x007D48D9+650985]
	sqlite3_dbdata_init [0x007C962C+605244]
	sqlite3_dbdata_init [0x007D586B+654971]
	(No symbol) [0x005AFEBC]
	(No symbol) [0x005A8428]
	(No symbol) [0x005A854D]
	(No symbol) [0x00595858]
	BaseThreadInitThunk 

In [341]:
df

,City,Interval,LCR,PGR,LCRPGR,City Definition data,Built-up data,Population data,Within city definition T1 Built-up (km²),"Within city definition T1 Population (*10,000)",Within city definition T1 LCPC (m²/person),Within city definition T2 Built-up (km²),"Within city definition T2 Population (*10,000)",Within city definition T2 LCPC (m²/person),% Change in LCPC LCPC (m²/person),Built-up T1,Built-up T2,% Change in urban infill
0,Stockholm,2015-2020,-0.128,0.021,-6.086,GHS-Pop 1km 2020,GHS-Built-S2 10m,GHS-Pop R2023A,268.169,185.107,144.872,141.378,205.785,68.702,-52.578,268.169,141.174,nan
1,Stockholm,2015-2020,-0.2,0.021,-9.503,GHS-Pop 1km 2020,WSF 2019 10m,GHS-Pop R2023A,336.092,185.107,181.566,123.622,205.785,60.074,-66.914,336.092,123.419,-63.278
2,Stockholm,2015-2020,0.004,0.021,0.181,GHS-Pop 1km 2020,GHS_Built_S,GHS-Pop R2023A,487.351,185.107,263.28,498.806,205.785,242.392,-7.934,487.351,496.698,1.918
3,Gothenburg,2015-2020,-0.154,0.016,-9.517,GHS-Pop 1km 2020,GHS-Built-S2 10m,GHS-Pop R2023A,160.136,70.179,228.182,74.061,76.118,97.298,-57.359,160.136,74.055,-53.755
4,Gothenburg,2015-2020,-0.205,0.016,-12.647,GHS-Pop 1km 2020,WSF 2019 10m,GHS-Pop R2023A,181.939,70.179,259.249,65.321,76.118,85.816,-66.898,181.939,65.302,-64.108
5,Gothenburg,2015-2020,0.006,0.016,0.347,GHS-Pop 1km 2020,GHS_Built_S,GHS-Pop R2023A,265.288,70.179,378.015,273.144,76.118,358.843,-5.072,265.288,272.863,2.855
6,Abu Dhabi,2015-2020,0.074,-0.024,-3.04,GHS-Pop 1km 2020,GHS-Built-S2 10m,GHS-Pop R2023A,129.119,145.913,88.491,188.821,129.452,145.862,64.834,129.119,NaN,44.093
7,Abu Dhabi,2015-2020,-0.136,-0.024,5.587,GHS-Pop 1km 2020,WSF 2019 10m,GHS-Pop R2023A,233.012,145.913,159.693,119.043,129.452,91.959,-42.415,233.012,NaN,-49.416
8,Abu Dhabi,2015-2020,0.02,-0.024,-0.802,GHS-Pop 1km 2020,GHS_Built_S,GHS-Pop R2023A,350.829,145.913,240.437,392.719,129.452,303.371,26.175,350.829,385.22,9.803
9,Accra,2015-2020,0.096,0.024,4.053,GHS-Pop 1km 2020,GHS-Built-S2 10m,GHS-Pop R2023A,567.391,498.698,113.775,921.311,562.569,163.769,43.941,567.391,916.644,61.554


In [23]:
# 关闭浏览器窗口
driver.quit()

如果程序中断，然后有子窗口开着，就可以跑下这个语句，让程序清空子窗口

In [334]:
# 关闭所有新窗口
for w in new_window_handle:      
    driver.switch_to.window(w)
    driver.close()
#swith to the origin main window
driver.switch_to.window(original_window_handle)

最完整爬取流程：循环城市，循环builtup数据集，循环population数据集。  
根据需要改代码，输出参考下面已输出的例子

In [325]:
##############################################################
# interate City
##############################################################
for index, city in enumerate(city_options[0:],start=0):
  if city.text=='':
        continue
  city_name = city.text
  print(f"Clicking on city {city_name} - index:{index}")

  # 点击城市
  city.click()

  ##############################################################
  # iterate builtup dataset
  ##############################################################
  builtup_element.click()
  builtup_options = WebDriverWait(driver, 10).until(
      EC.presence_of_all_elements_located((By.CLASS_NAME, 'goog-menuitem-content')))  

  for index, builtup in enumerate(builtup_options[0:],start=0):    
    if builtup.text=='':
        continue
    if builtup.text=='Own data':
        builtup.click()
        break
    print(f'Clicking on builtup {builtup.text} - index: {index}')
    builtup.click()

    ##############################################################
    # iterate Population dataset
    ##############################################################
    population_element.click()
    population_options = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'goog-menuitem-content'))
    )
      
    for index, population in enumerate(population_options[0:],start=0):      
      if population.text=='':
        continue   
      if population.text=='Own data':
         population.click()
         break     
            
      print(f'Clicking on  population {population.text} - index: {index}') 
      population.click()          

      ##############################################################
      # compute indicators
      ##############################################################
      #click the compute button
      compute_element=driver.find_elements(By.CSS_SELECTOR, '#content > div > div.ui-widget.ui-panel.stretch-horizontal.stretch-vertical > div > div.ui-widget.custom-width.ui-panel > div > div:nth-child(4) > div > div:nth-child(4) > div > div > div > div.ui-widget.ui-button > div')[0]
      compute_element.click()
      time.sleep(5)

      ##############################################################
      # Click population element
      ##############################################################
      population_element.click()            

    ##############################################################
    # Select Built-up dataset
    ##############################################################
    builtup_element.click()
    builtup_options = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, 'goog-menuitem-content')))

  ##############################################################
  # Select City
  ##############################################################
  city_element.click()
  city_options = WebDriverWait(driver, 10).until(
      EC.presence_of_all_elements_located((By.CLASS_NAME, 'goog-menuitem-content'))
  )

Clicking on city Stockholm - index:0
Clicking on builtup GHS-Built 30m - index: 48
Clicking on  population GHS-Pop 100m - index: 52
Clicking on  population WorldPop 100m - index: 53
Clicking on  population GPWv4 30 arc-second - index: 54
Clicking on builtup WSF evolution - index: 49
Clicking on  population GHS-Pop 100m - index: 52
Clicking on  population WorldPop 100m - index: 53
Clicking on  population GPWv4 30 arc-second - index: 54
Clicking on builtup GHS_Built_S 100m - index: 50
Clicking on  population GHS-Pop 100m - index: 52
Clicking on  population WorldPop 100m - index: 53


KeyboardInterrupt: 